# Purpose:
- To see if whisker feature angle discrimination affects clustering index
    - During naïve, learning, and expert sessions
## Hypothesis:
- Clustering index variability during naïve sessions are due to the variability in angle discriminability from sensory inputs (whisker features)
    - When sensory inputs discriminate angles better, S1 activity discriminates angles better.
## Method:
- Use LDA to discriminate the angles from whisker features.
- Look at the correlation between LDA results and clustering index
    - From different phases of learning

In [1]:
import numpy as np
import pandas as pd
from importlib import reload
from pathlib import Path
import matplotlib.pyplot as plt
import utils.clustering_index as ci
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

base_dir = Path(r'E:\TPM\JK\h5')
behavior_dir = Path(r'E:\TPM\JK\SoloData')
whisker_dir = Path(r'E:\TPM\JK\WhiskerVideo')
results_dir = base_dir / 'results'
load_dir = results_dir / 'pop_responses' / 'touch_before_answer'

expert_mice_df = pd.read_csv(base_dir / 'expert_mice.csv', index_col=0)
use_mice_df = expert_mice_df.loc[expert_mice_df['depth_matched'].astype(bool) & 
                                 ~expert_mice_df['processing_error'].astype(bool) &
                                 ((expert_mice_df.session_type == 'training') |
                                  (expert_mice_df.session_type.str.contains('test')))]
use_volume_df = use_mice_df.query('plane in [1, 5]')
use_volume_df.loc[:, 'volume'] = use_volume_df['plane'].apply(lambda x: 1 if x==1 else 2)
training_volume_df = use_volume_df.query('session_type == "training"')
remove_ind = training_volume_df.query('mouse==27 and session=="15"')
training_volume_df = training_volume_df.drop(remove_ind.index)
remove_ind = training_volume_df.query('mouse==36 and session=="9"')
training_volume_df = training_volume_df.drop(remove_ind.index)

mice = [25,27,30,36,39,52]
test_sessions = [[4,19], [3,8], [3,21], [1,17], [1,23], [3,21]]
naive_sessions = [10, 4, 11, 6, 6, 11]

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [4]:
mouse = 25
volume = 1
sessions = [int(s) for s in training_volume_df.query('mouse==@mouse and volume==@volume').session.unique()]
session = sessions[0]

In [7]:
session_wdir = whisker_dir / f'JK{mouse:03}S{session:02}'
whisker_filepath = session_wdir / f'JK{mouse:03}S{session:02}_whisker.pkl'
whisker_df = pd.read_pickle(whisker_filepath)

In [9]:
whisker_df.columns

Index(['amplitude', 'arcLength', 'kappaH', 'kappaV', 'midpoint', 'phase',
       'phi', 'poleAngle', 'poleDistance', 'poleMovingFrames', 'polePosition',
       'poleUpFrames', 'protractionSlide', 'protractionSlideByWhisking',
       'protractionTFchunks', 'protractionTFchunksByWhisking',
       'protractionTouchDuration', 'protractionTouchDurationByWhisking',
       'retractionSlide', 'retractionSlideByWhisking', 'retractionTFchunks',
       'retractionTFchunksByWhisking', 'retractionTouchDuration',
       'retractionTouchDurationByWhisking', 'theta', 'whisker_time',
       'whiskingStartFrames', 'mouse_name', 'session_name'],
      dtype='object')

In [10]:
whisker_df.iloc[0].protractionTFchunksByWhisking

[array([511., 512.]),
 array([522., 523., 524.]),
 array([530., 531., 532., 533.]),
 array(553.),
 array([566., 567., 568., 569., 570.]),
 array([595., 596., 597., 598.]),
 array([609., 610., 611., 612.]),
 array([621., 622., 623., 624., 625., 626., 627.]),
 array([639., 640., 641., 642.]),
 array([655., 656., 657., 658., 659., 660., 661., 662., 663., 664., 665.,
        666., 667.]),
 array([674., 675., 676., 677., 678., 679.]),
 array([686., 687., 688., 689., 690., 691., 692., 693.]),
 array([704., 705., 706., 707., 708., 709., 710., 711., 712.]),
 array([726., 727., 728.]),
 array([741., 742., 743., 744.]),
 array([756., 757., 758., 759., 760., 761., 762., 763., 764., 765., 766.,
        767., 768., 769., 770., 771., 772., 773., 774., 775., 776., 777.,
        778., 779., 780.]),
 array([818., 819., 820.]),
 array([825., 826., 827., 828., 829., 830., 831., 832., 833., 834., 835.,
        836., 837., 838., 839., 840.]),
 array([876., 877., 878., 879., 880., 881.]),
 array([905., 906.